<a href="https://colab.research.google.com/github/ysMarcos/projeto-python/blob/main/projeto_python.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Imports das Bibliotecas usadas

In [ ]:
#Imports
import requests as req
import hashlib
import pandas as pd
import sqlite3
import json
from google.colab import userdata

# Funções utilizadas

In [ ]:
#Funcao responsavel para encryptografar para md5 de acordo com a documentacao da api da Marvel
def hashToMD5(publicKey, privateKey):
  #Valor 1 eh o timestamp de exemplo usado na documentacao
  return hashlib.md5(str.encode(str(1) + str(privateKey) + str(publicKey)))

In [ ]:
# Funcao responsavel pelas requisicoes
def get(url: str, endpoint: str, headers: dict, params: dict):
  try:
    response = req.get(
        url = url+endpoint,
        headers = headers,
        params = params
      )
    return response.json()
  except Exception as e:
    print(e)

In [ ]:
#Funcoes responsaveis pela transformacao dos dados em um Dataframe e salvar o objeto passado
def saveToCSV(obj, csvPath):
  df = pd.DataFrame(obj)
  df.to_csv(csvPath + '.csv', encoding = 'utf-8', index = False, header = True)

def saveToSqlite(obj, tableName, dbPath='bd.db', uniqueColumns = None):
  try:
    df = pd.DataFrame(obj) if not isinstance(obj, pd.DataFrame) else obj
    with sqlite3.connect(dbPath) as con:
      df.to_sql(tableName, con, if_exists='append', index=False)
# Passei o trabalho pro banco remover as duplicatas em vez da aplicacao, ganhando performance
      if(uniqueColumns):
        uniqueClause = ", ".join(uniqueColumns)
        query = f"""
          DELETE FROM {tableName}
          WHERE rowid NOT IN (
              SELECT MIN(rowid)
              FROM {tableName}
              GROUP BY {uniqueClause}
          );
          """
        con.execute(query)
  except Exception as e:
    print(f"Erro: {e}")

# Endpoints

In [ ]:
#Variaveis que serao utilizadas em todos os endpoints da marvel
url = 'http://gateway.marvel.com/v1/public/'
publicKey = userdata.get('publicKey')
privateKey = userdata.get('privateKey')
securityHash = hashToMD5(publicKey, privateKey).hexdigest()
dbPath = userdata.get('dbPath')

headers = {
    'Accept': "*/*"
}

params = {
    "ts": 1,
    "apikey": publicKey,
    "hash": securityHash,
    "limit": 100
}

## Characters

In [ ]:
endpoint = 'characters'

result = get(url, endpoint, headers, params)['data']['results']
characters = [{
    "id": x['id'],
    "name": x["name"],
    "description": x["description"],
    "comics": ''.join([y["name"] for y in x["comics"]["items"]]),
    "events": ''.join([y["name"] for y in x["events"]["items"]])
  } for x in result]
saveToCSV(csvPath = endpoint, obj = characters)
saveToSqlite(characters, endpoint, dbPath, ['id'])

[{'id': 1011334, 'name': '3-D Man', 'description': '', 'modified': '2014-04-29T14:18:17-0400', 'thumbnail': {'path': 'http://i.annihil.us/u/prod/marvel/i/mg/c/e0/535fecbbb9784', 'extension': 'jpg'}, 'resourceURI': 'http://gateway.marvel.com/v1/public/characters/1011334', 'comics': {'available': 12, 'collectionURI': 'http://gateway.marvel.com/v1/public/characters/1011334/comics', 'items': [{'resourceURI': 'http://gateway.marvel.com/v1/public/comics/21366', 'name': 'Avengers: The Initiative (2007) #14'}, {'resourceURI': 'http://gateway.marvel.com/v1/public/comics/24571', 'name': 'Avengers: The Initiative (2007) #14 (SPOTLIGHT VARIANT)'}, {'resourceURI': 'http://gateway.marvel.com/v1/public/comics/21546', 'name': 'Avengers: The Initiative (2007) #15'}, {'resourceURI': 'http://gateway.marvel.com/v1/public/comics/21741', 'name': 'Avengers: The Initiative (2007) #16'}, {'resourceURI': 'http://gateway.marvel.com/v1/public/comics/21975', 'name': 'Avengers: The Initiative (2007) #17'}, {'resour

## Comics

In [ ]:
endpoint = 'comics'

result = get(url, endpoint, headers, params)['data']['results']
comics = [{
    "id": x["id"],
    "digitalId": x["digitalId"],
    "title": x["title"],
    "pageCount": x["pageCount"],
    "issueNumber": x["issueNumber"],
    "dates": [{y} for y in x["dates"]],
    "prices": [{y} for y in x["prices"]],
    "events": ''.join([y["name"] for y in x["events"]["items"]]),
    "creators": [y["name"] for y in x["creators"]]
  } for x in result]
saveToCSV(csvPath = endpoint, obj = comics)
saveToSqlite(comics, endpoint, dbPath, ["id"])

[{'id': 82967, 'digitalId': 0, 'title': 'Marvel Previews (2017)', 'issueNumber': 0, 'variantDescription': '', 'description': '', 'modified': '2019-11-07T08:46:15-0500', 'isbn': '', 'upc': '75960608839302811', 'diamondCode': '', 'ean': '', 'issn': '', 'format': '', 'pageCount': 112, 'textObjects': [], 'resourceURI': 'http://gateway.marvel.com/v1/public/comics/82967', 'urls': [{'type': 'detail', 'url': 'http://marvel.com/comics/issue/82967/marvel_previews_2017?utm_campaign=apiRef&utm_source=b4718540b964d64427a2e5434eda6274'}], 'series': {'resourceURI': 'http://gateway.marvel.com/v1/public/series/23665', 'name': 'Marvel Previews (2017 - Present)'}, 'variants': [{'resourceURI': 'http://gateway.marvel.com/v1/public/comics/82965', 'name': 'Marvel Previews (2017)'}, {'resourceURI': 'http://gateway.marvel.com/v1/public/comics/82970', 'name': 'Marvel Previews (2017)'}, {'resourceURI': 'http://gateway.marvel.com/v1/public/comics/82969', 'name': 'Marvel Previews (2017)'}, {'resourceURI': 'http://

## Events

In [ ]:
endpoint = "events"

result = get(url, endpoint, headers, params)["data"]["results"]
events = [{
    "id": x["id"],
    "title": x["title"],
    "description": x["description"],
    "start": x["start"],
    "end": x["end"],
    "comics": ''.join([y["name"] for y in x["comics"]["items"]]),
    "characters": ''.join([y["name"] for y in x["characters"]["items"]])
} for x in result]
saveToCSV(csvPath = endpoint, obj = events)
saveToSqlite(events, endpoint, dbPath, ["id"])

[{'id': 116, 'title': 'Acts of Vengeance!', 'description': 'Loki sets about convincing the super-villains of Earth to attack heroes other than those they normally fight in an attempt to destroy the Avengers to absolve his guilt over inadvertently creating the team in the first place.', 'resourceURI': 'http://gateway.marvel.com/v1/public/events/116', 'urls': [{'type': 'detail', 'url': 'http://marvel.com/comics/events/116/acts_of_vengeance?utm_campaign=apiRef&utm_source=b4718540b964d64427a2e5434eda6274'}, {'type': 'wiki', 'url': 'http://marvel.com/universe/Acts_of_Vengeance!?utm_campaign=apiRef&utm_source=b4718540b964d64427a2e5434eda6274'}], 'modified': '2013-06-28T16:31:24-0400', 'start': '1989-12-10 00:00:00', 'end': '2008-01-04 00:00:00', 'thumbnail': {'path': 'http://i.annihil.us/u/prod/marvel/i/mg/9/40/51ca10d996b8b', 'extension': 'jpg'}, 'creators': {'available': 98, 'collectionURI': 'http://gateway.marvel.com/v1/public/events/116/creators', 'items': [{'resourceURI': 'http://gatewa